### 二叉搜索树
&emsp;&emsp;二叉搜索树上的基本操作所花费的时间与这棵树的高度成正比，对于有n个节点的一颗完全二叉树来说，这些操作的最坏运行时间为O(lgn),然而，如果这颗树是一条n个节点组成的线性链表，那么同样的操作就要花费O(n)的最坏运行时间。

#### 1.什么是二叉搜索树
&emsp;&emsp;一棵树可以使用一个链表的数据结构来表示，其中每一个节点就是一个对象，除了key关键字外，还包含属性left,right,parent,他们分别指向节点的左孩子，右孩子和双亲，如果某个孩子节点和父节点不存在，则相应的属性值为None，根节点是树中唯一父指针为None的节点。二叉搜索树中的关键字总是以满足二叉搜索树性质的方式来存储:

&emsp;&emsp;**设x是二叉搜索树中的一个节点，如果y是x左子树中的一个节点，那么y.key <= x.key，如果y是x右子树中的一个节点，那么y.key >= x.key。**    
&emsp;&emsp;二叉搜索树性质允许我们通过一个简单的递归算法来按序输出二叉搜索树中的所有关键子，这种算法成为中序遍历，命名的原因是输出的子树根的关键字位于起左子树的关键字和右子树的关键字之间。

In [1]:
# 中序遍历的递归算法
def inorder_walk(x):
    if x:
        inorder_walk(x.left)
        print(x.key)
        inorder_walk(x.right)

#### 2.查询二叉搜索树
&emsp;&emsp;经常需要查找一个存储在二叉搜索树中的关键字，除了search操作之外，二叉搜索树还能支持minmum,maximum,successor,predecessor的查询操作。这些操作的的时间为O(h)。
##### 2.1. 查找
&emsp;&emsp;在一颗二叉搜索树中查找一个具有给定关键字的节点，输入一个指向树根的指针和一个关键字key,如果这个节点存在，返回一个指向关键字为key的节点的指针，否则返回None

In [2]:
# 递归的方式寻找
def rec_search(x,key):
    if x is None or x.key == key:
        return x
    if key < x.key:
        return rec_search(x.left,key)
    else:
        return rec_search(x.right,key)
    
# 迭代的方式寻找
def iter_search(x,key):
    while x and key != x.key:
        if key < x.key:
            x = x.left
        else:
            x = x.right
    return x

##### 2.2. 最大关键字元素和最小关键字元素
&emsp;&emsp;通过从树根开始沿着left孩子节点直到遇到一个None,总能在一颗二叉搜索树中找到一个元素，寻找最小关键字，一定是在最左边的节点，寻找最大关键字，一定是在最右边的节点。

In [3]:
# 寻找最小关键字节点
def tree_minimum(x):
    while x.left:
        x = x.left
    return x

# 寻找最大关键字节点
def tree_maximum(x):
    while x.right:
        x = x.right
    return x

&emsp;&emsp;这两个过程在一颗高度为h的树上均能在O(h)的时间内执行完成。
##### 2.3. 后继和前驱
&emsp;&emsp;给定一颗二叉搜索树中的一个节点，有时候需要按中序遍历的次序查找它的后继节点，如果所有的关键字互不相同，则一个节点x的后继是大于x.key的最小关键字的节点，前驱节点是小于x.key的最大关键字的节点。
&emsp;&emsp;**后继**
&emsp;&emsp;如果节点x的右子树存在，那么x的后继节点应该是x的右子树中的最左节点；如果节点x的右子树为空并有一个后继y，那么y就是x的有左孩子的最底层祖先，并且它也是x的一个祖先，这个仔细想想怎么回事,此时x没有右子树，那么需要向上寻找，如果x是x的父节点的左子树，那么x的父节点就是x的后继节点，如果x是x的父节点的右子树，那么说明x的父节点不是，需要继续向上寻找，那么什么时候是，直到根节点或者当前节点是父节点的左孩子。举例说明如下:
                      15
                    /    \
                  6       18
                /  \     /  \
               3    7   17   20
             /  \    \
            2   4    13
                     /
                    9 
&emsp;&emsp;对于节点9,它没有右孩子，然后它恰好是父节点的左孩子，因此13就是它的后继节点；对于13节点，它没有右孩子，同时它是父节点的右子树，说明13节点是某个根节点的右子树，那么向上寻找找到一个节点，使得该节点是父节点的左子树，那么该节点的父子树就是后继节点，对于节点13，一直向上查找，找到6，发现6节点是15节点的左孩子，最后返回15节点


In [4]:
# 寻找后继节点
def tree_successor(x):
    if x.right:
        return tree_minimum(x.right)
    p = x.parent
    while y and x == p.right:
        x = y
        y = y.parent
    return y

#### 3.插入和删除
&emsp;&emsp;插入和删除操作会引起二叉搜索树表示的动态结构的变化，一定要修改数据结构来反映这个变化，但是修改要保持二叉搜索树性质的成立，插入节点的树修改要简单些，而删除的处理有些复杂。
##### 3.1. 插入
&emsp;&emsp;要将一个新值v插入到一颗二叉搜索树T中，该过程以节点z作为输入，其中z.key = v,z.left = None,z.right = None ，这个过程需要修改T和z的某些属性，来把z插入到树中的相应位置上。

In [5]:
def tree_insert(T,z):
    # y存储的是最后的叶子节点
    y = None
    x = T.root
    while x:
        y = x
        if z.key < x.key:
            x = x.left
        else:
            x = x.right
    z.parent = y
    if y is None:
        T.root = z
    elif z.key < y.key:
        y.left = z
    else:
        y.right = z

##### 3.2. 删除
&emsp;&emsp;二叉树的删除算法:(这个自己看过好多次，就是记不住)

        算法思想：
        1.如果z没有孩子节点，简单的将其删除，并修改z的父节点，用None替换
        2.如果z只有一个孩子节点,将z的孩子提升到树中z的位置，并修改z的父节点，用z的孩子节点替换z
        3.如果z有两个节点，那么寻找z的后继节点y(一定在z的右子树中)，找到后
           i.如果y是z的右孩子，那么用y替换z节点，并保留y的右孩子（y一定没有左孩子）
           ii.否则，先用y的右孩子替换y,然后在用y替换z
        转换为如下四种情况：
        1.如果z没有左孩子，那么用z的右孩子替换z，
        2.如果z仅有一个孩子且为左孩子，那么用z的左孩子替换z
        3.如果z有左孩子和右孩子，需要查找z的后继节点y （y位于z的右子树中并且没有左孩子）
           i.如果y是z的右孩子，用y替换z，并仅保留z的右孩子
           ii.否则，先用y的右孩子替换y，然后在用y替换z
           
![delete](./pic/tree/tree_delete.jpg)

#### 4.树的完整代码

In [6]:
#!/usr/bin/env python
# coding=utf-8
'''
    author:fengjiaxin
    desc:二叉搜索树
'''
__author__ = 'fengjiaxin'

class Tree():
    def __init__(self,root=None):
        self.root = root

class Node():
    '''
        节点元素定义
    '''
    def __init__(self,key,parent=None,left = None,right = None):
        self.key = key
        self.parent = parent
        self.left = left
        self.right = right


def inOrderWalkNode(node):
    '''
        中序遍历二叉搜索树，从小到大输出元素
    '''
    if node:
        inOrderWalkNode(node.left)
        print(node.key,end=' ')
        inOrderWalkNode(node.right)

def inOrderWalk(tree):
    inOrderWalkNode(tree.root)
    print()


def cursion_tree_search(tree,x):
    '''
        二叉树搜索，递归版本
    '''
    root = tree.root
    if not root or x == root.key:
        return root
    if x < root.key:
        return cursion_tree_search(root.left,x)
    else:
        return cursion_tree_search(root.right,x)

def iterative_tree_search(tree,x):
    '''
        二叉搜索树，迭代版本
    '''
    root = tree.root
    while root and x != root.key:
        if x < root.key:
            root = root.left
        else:
            root = root.right
    return root

def tree_minmum(node):
    '''
        返回以node节点为根结点的树的最小节点
    '''
    while node.left:
        node = node.left
    return node

def tree_maximum(node):
    '''
        返回以node节点为根结点的树的最大节点
    '''
    while node.right:
        node = node.right
    return node

'''
    查找节点的后继和前驱
    一个节点x的后继是大于x.key的最小关键字的节点
    一个节点x的前驱是小于x.key的最大关键字的节点


                   15
                /       \
            6               18
          /   \           /    \
        3       7       17      20
       / \        \
      2   4        13
                   /
                  9
'''

def nextNode(node):
    '''
        查找x节点的后继节点
        分两种情况：
        1.如果x节点的右子树非空，那么x的后继节点就是x的右子树的最左节点(7的后继节点就是9)
        2.如果x节点的右子树为空并有一个后继y，那么y就是x的有左孩子的最底层祖先，并且y也是x的一个祖先
          (13的后继节点是15，17的后继节点是18)
    '''
    if node.right:
        return tree_minmum(node.right)
    p = node.p
    while p and p.right == node:
        node = p
        p = node.parent
    return p

def preNode(node):
    '''
        查找x节点的前驱节点
        分两种情况:
        1.如果x节点的左子树非空，那么x的前驱节点就是x的左子树的最右节点（13的前驱节点是9）
        2.如果x节点的左子树为空并有一个前驱y，那么y就是x的有右孩子的最底层祖先，并且y也是x的一个祖先
          （17的前驱节点是15，13的前驱节点是7）
    '''
    if node.left:
        return tree_maximum(node.left)
    p = node.p
    while p and p.left == node:
        node = p
        p = node.parent
    return p

def treeInsert(tree,node):
    '''
        二叉搜索树的插入算法
        从树根开始，指针x记录一条向下的路径，遍历指针y作为x的双亲，最后x的位置就是要插入node节点的位置
    '''
    y = None
    root = tree.root
    while root:
        y = root
        if node.key < root.key:
            root = root.left
        else:
            root = root.right
    node.parent = y
    # empty tree
    if y is None:
        tree.root = node
    elif node.key < y.key:
        y.left = node
    else:
        y.right = node


def transplant(Tree,u,v):
    '''
        用一颗以v为根的子树来替换一颗以u为根的子树时，节点u的双亲变为节点v的双亲
        并且v成为u的双亲的相应孩子
    '''
    if u.parent is None:
        Tree.root = v
    elif u == u.parent.left:
        u.parent.left = v
    else:
        u.parent.right = v
    if v:
        v.parent = u.parent

def treeDelete(tree,z):
    '''
        二叉树的删除算法:(这个自己看过好多次，就是记不住)
        算法思想：
        1.如果z没有孩子节点，简单的将其删除，并修改z的父节点，用None替换
        2.如果z只有一个孩子节点,将z的孩子提升到树中z的位置，并修改z的父节点，用z的孩子节点替换z
        3.如果z有两个节点，那么寻找z的后继节点y(一定在z的右子树中)，找到后
           i.如果y是z的右孩子，那么用y替换z节点，并保留y的右孩子（y一定没有左孩子）
           ii.否则，先用y的右孩子替换y,然后在用y替换z
        转换为如下四种情况：
        1.如果z没有左孩子，那么用z的右孩子替换z
        2.如果z仅有一个孩子且为左孩子，那么用z的左孩子替换z
        3.如果z有左孩子和右孩子，需要查找z的后继节点y （y位于z的右子树中并且没有左孩子）
           i.如果y是z的右孩子，用y替换z，并仅保留z的右孩子
           ii.否则，先用y的右孩子替换y，然后在用y替换z

    '''
    if z.left is None:
        transplant(tree,z,z.right)
    elif z.right is None:
        transplant(tree,z,z.left)
    else:
        y = tree_minmum(z.right)
        if y.parent != z:
            transplant(tree,y,y.right)
            y.right = z.right
            y.right.parent = y
        transplant(tree,z,y)
        y.left = z.left
        y.left.parent = y

'''
    自己总结一下二叉树删除的全过程以及思考
    首先被要删除的节点分三种情况
    1.该节点是叶子节点
    2.该节点只有一个子节点
    3.该节点有左右两个子节点
    继续分析，其实该节点只有一个节点是分两种情况的，该节点只有一个节点是左孩子的，该节点只有一个孩子是右孩子的
    所以真实情况是四种，其实可以按照叶子节点是不是空重新总结为3种情况
    1.如果该节点的左子树为空，那么右子树是否为空都不重要，直接将该节点的右子节点替换该节点
    2.否则，如果该节点的右子树为空，那么将该节点的左子节点替换该节点
    3.否则(左右子树都不为空)，这种需要考虑该节点的后继节点了，这种不能直接替换的原因是该节点有左子树，需要处理左子树
      i.如果该节点的后继节点是该节点的右子树，那么用y替换z,同时还需要处理z的原来左子树，需要处理的有y的左子树，以及y的左子树的parent
      ii.如果该节点的后继节点不是该节点的右子树，首先明确一下思路，最后肯定是要后继节点替换原来的节点，但是后继节点y在z的右子树中，
           现在需要做的就是将树分开，分成两个树，一个是原来的树，另一个树需要将后继节点y作为根结点，然后在用y节点替换z节点，同时
           处理z节点的原来左子树
    

'''

if __name__ == '__main__':
    ss = [4, 23, 65, 22, 12, 3, 7, 1, 256, 34, 27]
    tree = Tree()
    for i in ss:
        treeInsert(tree, Node(i))
    n = Node(26)
    inOrderWalk(tree)
    treeInsert(tree, n)
    inOrderWalk(tree)
    treeDelete(tree, n)
    inOrderWalk(tree)

1 3 4 7 12 22 23 27 34 65 256 
1 3 4 7 12 22 23 26 27 34 65 256 
1 3 4 7 12 22 23 27 34 65 256 


#### 5.tree leetcode例题

| 基础        | 例题                                                          | 描述                   |
|-----------|------------------------------------------------------------|-----------------------|
| 144       | Binary Tree Preorder Traversal                             | preorder              |
| 94        | Binary Tree Inorder Traversal                              | Inorder               |
| 145       | Binary Tree Postorder Traversal                            | postorder             |
| 102       | Binary Tree Level Order Traversal                          | DFS \+ BFS            |
 |
| Preorder  |
| 100       | Same Tree                                                  | preorder              |
| 101       | Symmetric Tree                                             | preorder              |
| 226       | Invert Binary Tree                                         | preorder \+ BFS       |
| 257       | Binary Tree Paths                                          | preorder              |
| 112       | Path Sum                                                   | preorder              |
| 113       | Path Sum II                                                | preorder              |
| 129       | Sum Root to Leaf Numbers                                   | preorder              |
| 298       | Binary Tree Longest Consecutive Sequence                   | preorder              |
| 111       | Minimum Depth of Binary Tree                               | preorder              |
 |
| Postorder |
| 104       | Maximum Depth of Binary Tree                               | postorder             |
| 110       | Balanced Binary Tree                                       | postorder             |
| 124       | Binary Tree Maximum Path Sum                               | postorder             |
| 250       | Count Univalue Subtrees                                    | postorder             |
| 366       | Find Leaves of Binary Tree                                 | postorder             |
| 337       | House Robber III                                           | postorder \+ preorder |
 |
| BFS       |
| 107       | Binary Tree Level Order Traversal II                       | BFS                   |
| 103       | Binary Tree Zigzag Level Order Traversal                   | BFS                   |
| 199       | Binary Tree Right Side View                                | BFS \+ preorder       |
 |
| BST       |
| 98        | Validate Binary Search Tree                                | preorder              |
| 235       | Lowest Common Ancestor of a Binary Search Tree             | preorder              |
| 236       | Lowest Common Ancestor of a Binary Tree                    | postorder             |
| 108       | Convert Sorted Array to Binary Search Tree                 | binary search         |
| 109       | Convert Sorted List to Binary Search Tree                  | binary search         |
| 173       | Binary Search Tree Iterator                                | inorder               |
| 230       | Kth Smallest Element in a BST                              | inorder               |
| 297       | Serialize and Deserialize Binary Tree                      | BFS                   |
| 285       | Inorder Successor in BST                                   | inorder               |
| 270       | Closest Binary Search Tree Value                           | preorder              |
| 272       | Closest Binary Search Tree Value II                        | inorder               |
| 99        | Recover Binary Search Tree                                 | inorder               |
 |
| 重要程度      |
| 156       | Binary Tree Upside Down                                    | 很少考                   |
| 114       | Flatten Binary Tree to Linked List                         | 很少考                   |
| 255       | Verify Preorder Sequence in Binary Search Tree             | 很少考                   |
| 333       | Largest BST Subtree                                        | 很少考                   |
| 222       | Count Complete Tree Nodes                                  | 很少考                   |
| 105       | Construct Binary Tree from Preorder and Inorder Traversal  | 很少考                   |
| 106       | Construct Binary Tree from Inorder and Postorder Traversal | 很少考                   |
| 116       | Populating Next Right Pointers in Each Node                | 重要                    |
| 117       | Populating Next Right Pointers in Each Node II             | 重要                    |
| 314       | Binary Tree Vertical Order Traversal                       | 重要                    |
| 96        | Unique Binary Search Trees                                 | 重要                    |
| 95        | Unique Binary Search Trees II                              | 很少考                   |
| 331       | Verify Preorder Serialization of a Binary Tree             | 很少考                   |

